<a href="https://colab.research.google.com/github/CodeandoMexico/hacking-civico/blob/master/notebooks/05_Exploracion_de_datos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<p align="center">
<img src="http://codeandomexico.org/resources/img/codeandomexico.png" width="500" alt="Codeando México"><br>
<a href="http://www.codeandomexico.org/" target="_blank"><img src="https://img.shields.io/badge/website-CodeandoMexico-00D88E.svg"></a>
<a href="http://slack.codeandomexico.org/" target="_blank"><img src="https://img.shields.io/badge/slack-CodeandoMexico-EC0E4F.svg"></a>
</p>
<!-- __ -->

# Curso de Datos Abiertos y Hacking Cívico

Este curso tiene como objetivo habilitar las capacidades de la ciudadanía y los servidores públicos en el uso y generación de datos abiertos para el bien común. Puedes encontrar más información [aquí](https://github.com/CodeandoMexico/hacking-civico).

# Exploración de datos: COVID-19 en México

En este cuaderno aprenderemos algunas bases de exploración de datos. Los datos a utilizar son los datos abiertos sobre COVID-19 en México, los cuales pueden ser accedidos directamente desde el portal: https://www.gob.mx/salud/documentos/datos-abiertos-152127

Del portal de datos abiertos podemos encontrar dos enlaces, uno para descargar el conjunto de datos y otro para descargar el diccionario; así que podemos automatizar el proceso de descarga y descompresión de la información.

Para obtener los datos, sólo basta que ejecutes la siguiente línea de código, que contiene el comando de descarga de la información.

In [1]:
# Para descargar el conjunto de datos
!wget http://datosabiertos.salud.gob.mx/gobmx/salud/datos_abiertos/datos_abiertos_covid19.zip

# Para descargar el diccionario de los datos
!wget http://epidemiologia.salud.gob.mx/gobmx/salud/datos_abiertos/diccionario_datos_covid19.zip

# Descomprimimos ambas carpetas y movemos archivos
!unzip datos_abiertos_covid19.zip
!unzip diccionario_datos_covid19.zip
!mv ./diccionario_datos_covid19/*.xlsx .
!mv *COVID19MEXICO.csv COVID19MEXICO.csv

# Eliminamos las carpetas comprimidas
!rm -rf diccionario_datos_covid19
!rm datos_abiertos_covid19.zip
!rm diccionario_datos_covid19.zip

--2020-09-05 00:46:26--  http://datosabiertos.salud.gob.mx/gobmx/salud/datos_abiertos/datos_abiertos_covid19.zip
Resolving datosabiertos.salud.gob.mx (datosabiertos.salud.gob.mx)... 187.210.186.146
Connecting to datosabiertos.salud.gob.mx (datosabiertos.salud.gob.mx)|187.210.186.146|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 29451718 (28M) [application/x-zip-compressed]
Saving to: ‘datos_abiertos_covid19.zip’

datos_abiertos_covi 100%[===================>]  28.09M  19.9MB/s    in 1.4s    

2020-09-05 00:46:27 (19.9 MB/s) - ‘datos_abiertos_covid19.zip’ saved [29451718/29451718]

--2020-09-05 00:46:28--  http://epidemiologia.salud.gob.mx/gobmx/salud/datos_abiertos/diccionario_datos_covid19.zip
Resolving epidemiologia.salud.gob.mx (epidemiologia.salud.gob.mx)... 187.191.75.207
Connecting to epidemiologia.salud.gob.mx (epidemiologia.salud.gob.mx)|187.191.75.207|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://ep

Hasta este punto, si abres la barra lateral izaquiera, deberías ser capaz de encontrar los datos:

- `COVID19MEXICO.csv`
- `Catalogos_0412.xlsx`
- `Descriptores_0419.xlsx`

👈🏼

## Carga

Ahora que tenemos los datos, procederemos a cargarlos. Para cargar los datos, primero importaremos algunos paquetes de Python, pues su funcionalidad nos será muy util.

In [2]:
import statistics

import pandas as pd
import numpy as np

In [3]:
# Especificamos el nombre del archivo por abrir
nombre_archivo = 'COVID19MEXICO.csv'

# Utilizamos la función de Pandas para cargar un archivo CSV
data = pd.read_csv(nombre_archivo, encoding='latin-1')

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (33) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Mostramos la cabeza (primeros elementos del archivo).

In [4]:
data.head()

,FECHA_ACTUALIZACION,ID_REGISTRO,ORIGEN,SECTOR,ENTIDAD_UM,SEXO,ENTIDAD_NAC,ENTIDAD_RES,MUNICIPIO_RES,TIPO_PACIENTE,FECHA_INGRESO,FECHA_SINTOMAS,FECHA_DEF,INTUBADO,NEUMONIA,EDAD,NACIONALIDAD,EMBARAZO,HABLA_LENGUA_INDIG,DIABETES,EPOC,ASMA,INMUSUPR,HIPERTENSION,OTRA_COM,CARDIOVASCULAR,OBESIDAD,RENAL_CRONICA,TABAQUISMO,OTRO_CASO,RESULTADO,MIGRANTE,PAIS_NACIONALIDAD,PAIS_ORIGEN,UCI
0,2020-09-03,097d14,2,3,2,2,2,2,2,1,2020-06-10,2020-06-10,9999-99-99,97,2,27,1,97,2,2,2,2,2,2,2,2,2,2,2,1,1,99,MÃ©xico,99,97
1,2020-09-03,1b7adb,2,4,2,2,12,2,4,2,2020-04-29,2020-04-15,2020-04-30,2,1,64,1,97,2,1,2,2,2,1,2,2,2,2,2,99,1,99,MÃ©xico,99,2
2,2020-09-03,11daaa,2,3,27,1,27,27,15,1,2020-05-26,2020-05-26,9999-99-99,97,2,32,1,2,99,2,2,2,2,2,2,2,2,2,2,1,1,99,MÃ©xico,99,97
3,2020-09-03,17c9eb,2,3,27,1,27,27,13,1,2020-06-05,2020-05-31,9999-99-99,97,2,54,1,2,2,1,2,2,2,1,2,2,2,2,2,1,1,99,MÃ©xico,99,97
4,2020-09-03,14f12c,2,4,17,1,3,17,7,1,2020-04-02,2020-04-01,9999-99-99,97,2,44,1,2,2,2,2,2,2,2,1,2,2,2,2,1,1,99,MÃ©xico,99,97


Ahora procedemos a cargar el archivo con los descriptores.

In [5]:
nombre_diccionario = 'Descriptores_0419.xlsx'
diccionario = pd.read_excel(nombre_diccionario)

In [6]:
diccionario

,Nº,NOMBRE DE VARIABLE,DESCRIPCIÓN DE VARIABLE,FORMATO O FUENTE
0,1,FECHA_ACTUALIZACION,"La base de datos se alimenta diariamente, esta...",AAAA-MM-DD
1,2,ID_REGISTRO,Número identificador del caso,TEXTO
2,3,ORIGEN,La vigilancia centinela se realiza a través de...,CATÁLOGO: ORIGEN ...
3,4,SECTOR,Identifica el tipo de institución del Sistema ...,CATÁLOGO: SECTOR ...
4,5,ENTIDAD_UM,Identifica la entidad donde se ubica la unidad...,CATALÓGO: ENTIDADES
5,6,SEXO,Identifica al sexo del paciente.,CATÁLOGO: SEXO
6,7,ENTIDAD_NAC,Identifica la entidad de nacimiento del paciente.,CATALÓGO: ENTIDADES
7,8,ENTIDAD_RES,Identifica la entidad de residencia del paciente.,CATALÓGO: ENTIDADES
8,9,MUNICIPIO_RES,Identifica el municipio de residencia del paci...,CATALÓGO: MUNICIPIOS
9,10,TIPO_PACIENTE,Identifica el tipo de atención que recibió el ...,CATÁLOGO: TIPO_PACIENTE


Finalmente, cargamos el rachivo de catálogos.

In [7]:
nombre_cat = 'Catalogos_0412.xlsx'
sheets = ['ORIGEN', 'SECTOR', 'SEXO', 'TIPO_PACIENTE', 'SI_NO', 'NACIONALIDAD', 'RESULTADO', 'de ENTIDADES', 'MUNICIPIOS']
catalogues = {}
for sheet in sheets:
  catalogue = pd.read_excel(nombre_cat, sheet_name='Catálogo ' + sheet)
  catalogue.columns = catalogue.columns.str.replace(' ', '_')
  catalogue.columns = catalogue.columns.str.replace('.', '')
  catalogue.columns = catalogue.columns.str.lower()
  new_catalogue = {sheet: catalogue}
  catalogues.update(new_catalogue)

In [8]:
catalogues

{'MUNICIPIOS':       clave_municipio        municipio  clave_entidad
 0                   1   AGUASCALIENTES              1
 1                   2         ASIENTOS              1
 2                   3         CALVILLO              1
 3                   4            COSÍO              1
 4                   5      JESÚS MARÍA              1
 ...               ...              ...            ...
 2496              999  NO ESPECIFICADO             32
 2497              999  NO ESPECIFICADO             36
 2498              997        NO APLICA             97
 2499              998        SE IGNORA             98
 2500              999  NO ESPECIFICADO             99
 
 [2501 rows x 3 columns], 'NACIONALIDAD':    clave      descripción
 0      1         MEXICANA
 1      2       EXTRANJERA
 2     99  NO ESPECIFICADO, 'ORIGEN':    clave      descripción
 0      1            USMER
 1      2   FUERA DE USMER
 2     99  NO ESPECIFICADO, 'RESULTADO':   unnamed:_0              unnamed:_1
 0    

In [9]:
catalogues['SEXO']

,clave,descripción
0,1,MUJER
1,2,HOMBRE
2,99,NO ESPECIFICADO


##Perfilamiento

###Descripción General

In [10]:
# dimensión de los datos
print('Número de filas: {}'.format(data.shape[0]))
print('Número de columnas: {}'.format(data.shape[1]))

Número de filas: 1393033
Número de columnas: 35


In [11]:
# columnas (variables)
data.columns

Index(['FECHA_ACTUALIZACION', 'ID_REGISTRO', 'ORIGEN', 'SECTOR', 'ENTIDAD_UM',
       'SEXO', 'ENTIDAD_NAC', 'ENTIDAD_RES', 'MUNICIPIO_RES', 'TIPO_PACIENTE',
       'FECHA_INGRESO', 'FECHA_SINTOMAS', 'FECHA_DEF', 'INTUBADO', 'NEUMONIA',
       'EDAD', 'NACIONALIDAD', 'EMBARAZO', 'HABLA_LENGUA_INDIG', 'DIABETES',
       'EPOC', 'ASMA', 'INMUSUPR', 'HIPERTENSION', 'OTRA_COM',
       'CARDIOVASCULAR', 'OBESIDAD', 'RENAL_CRONICA', 'TABAQUISMO',
       'OTRO_CASO', 'RESULTADO', 'MIGRANTE', 'PAIS_NACIONALIDAD',
       'PAIS_ORIGEN', 'UCI'],
      dtype='object')

In [12]:
# limpieza del nombre de las columnas, remover espacios, carácteres especiales y pasar a minúsculas
data.columns = data.columns.str.replace(' ', '_')
data.columns = data.columns.str.replace('.', '')
data.columns = data.columns.str.lower()

In [13]:
# calcular número de valores nulos por columna
for col in data.columns:
  print('Valores nulos en "{}": {}'.format(col,data[col].isna().sum()))

Valores nulos en "fecha_actualizacion": 0
Valores nulos en "id_registro": 0
Valores nulos en "origen": 0
Valores nulos en "sector": 0
Valores nulos en "entidad_um": 0
Valores nulos en "sexo": 0
Valores nulos en "entidad_nac": 0
Valores nulos en "entidad_res": 0
Valores nulos en "municipio_res": 0
Valores nulos en "tipo_paciente": 0
Valores nulos en "fecha_ingreso": 0
Valores nulos en "fecha_sintomas": 0
Valores nulos en "fecha_def": 0
Valores nulos en "intubado": 0
Valores nulos en "neumonia": 0
Valores nulos en "edad": 0
Valores nulos en "nacionalidad": 0
Valores nulos en "embarazo": 0
Valores nulos en "habla_lengua_indig": 0
Valores nulos en "diabetes": 0
Valores nulos en "epoc": 0
Valores nulos en "asma": 0
Valores nulos en "inmusupr": 0
Valores nulos en "hipertension": 0
Valores nulos en "otra_com": 0
Valores nulos en "cardiovascular": 0
Valores nulos en "obesidad": 0
Valores nulos en "renal_cronica": 0
Valores nulos en "tabaquismo": 0
Valores nulos en "otro_caso": 0
Valores nulos 

In [14]:
# tipos de datos
data.dtypes

fecha_actualizacion    object
id_registro            object
origen                  int64
sector                  int64
entidad_um              int64
sexo                    int64
entidad_nac             int64
entidad_res             int64
municipio_res           int64
tipo_paciente           int64
fecha_ingreso          object
fecha_sintomas         object
fecha_def              object
intubado                int64
neumonia                int64
edad                    int64
nacionalidad            int64
embarazo                int64
habla_lengua_indig      int64
diabetes                int64
epoc                    int64
asma                    int64
inmusupr                int64
hipertension            int64
otra_com                int64
cardiovascular          int64
obesidad                int64
renal_cronica           int64
tabaquismo              int64
otro_caso               int64
resultado               int64
migrante                int64
pais_nacionalidad      object
pais_orige

###Limpieza

In [15]:
# convertimos a fecha algunas columnas
def cast_datetime_cols(df):
    timestamp_cols = [col for col in df.columns if "fecha" in col]
    df[timestamp_cols] = df[timestamp_cols].apply(lambda date_col: pd.to_datetime(date_col, errors="coerce"), axis=0)
    return df

data = cast_datetime_cols(data)

In [16]:
data.head()

,fecha_actualizacion,id_registro,origen,sector,entidad_um,sexo,entidad_nac,entidad_res,municipio_res,tipo_paciente,fecha_ingreso,fecha_sintomas,fecha_def,intubado,neumonia,edad,nacionalidad,embarazo,habla_lengua_indig,diabetes,epoc,asma,inmusupr,hipertension,otra_com,cardiovascular,obesidad,renal_cronica,tabaquismo,otro_caso,resultado,migrante,pais_nacionalidad,pais_origen,uci
0,2020-09-03,097d14,2,3,2,2,2,2,2,1,2020-06-10,2020-06-10,NaT,97,2,27,1,97,2,2,2,2,2,2,2,2,2,2,2,1,1,99,MÃ©xico,99,97
1,2020-09-03,1b7adb,2,4,2,2,12,2,4,2,2020-04-29,2020-04-15,2020-04-30,2,1,64,1,97,2,1,2,2,2,1,2,2,2,2,2,99,1,99,MÃ©xico,99,2
2,2020-09-03,11daaa,2,3,27,1,27,27,15,1,2020-05-26,2020-05-26,NaT,97,2,32,1,2,99,2,2,2,2,2,2,2,2,2,2,1,1,99,MÃ©xico,99,97
3,2020-09-03,17c9eb,2,3,27,1,27,27,13,1,2020-06-05,2020-05-31,NaT,97,2,54,1,2,2,1,2,2,2,1,2,2,2,2,2,1,1,99,MÃ©xico,99,97
4,2020-09-03,14f12c,2,4,17,1,3,17,7,1,2020-04-02,2020-04-01,NaT,97,2,44,1,2,2,2,2,2,2,2,1,2,2,2,2,1,1,99,MÃ©xico,99,97


In [17]:
# unimos los datos con los catalogos para tener mejor entendimiento de ellos
data = (
    # unimos con el catálogo MUNICIPIOS y ENTIDADES
    data.merge(catalogues['de ENTIDADES'], how='left', left_on='entidad_res', right_on='clave_entidad').
    drop(columns=['entidad_res', 'abreviatura']).
    rename(columns={'entidad_federativa':'entidad_res'}).
    merge(catalogues['MUNICIPIOS'], how='left', left_on=['clave_entidad','municipio_res'], right_on=['clave_entidad','clave_municipio']).
    drop(columns=['municipio_res', 'clave_entidad', 'clave_municipio']).
    rename(columns={'municipio':'municipio_res'})
)

In [18]:
# unimos con el catálogo de ENTIDADES
entidades_cols = [col for col in data.columns if "entidad" in col and col != 'entidad_res']
for col in entidades_cols:
    data = (
        data.merge(catalogues['de ENTIDADES'], how='left', left_on=col, right_on='clave_entidad').
        drop(columns=[col, 'abreviatura', 'clave_entidad']).
        rename(columns={'entidad_federativa':col})
    )

In [19]:
# unimos con el catálogo SI_NO
sino_cols = ['intubado', 'neumonia', 'embarazo', 'habla_lengua_indig', 'diabetes', 'epoc', 'asma', 'inmusupr',
             'hipertension', 'otra_com', 'cardiovascular', 'obesidad', 'renal_cronica', 'tabaquismo', 
             'otro_caso', 'migrante', 'uci']
for col in sino_cols:
    data = (
        data.merge(catalogues['SI_NO'], how='left', left_on=col, right_on='clave').
        drop(columns=[col, 'clave']).
        rename(columns={'descripción':col})
    )

> **Nota:**
>
> Si ejecutaste una versión previa del cuaderno, notarás que en la siguiente sección aparecía un error. A continuación dejamos como un pequeño paréntesis un poco más de limpieza sobre el cuaderno.

1. Corregimos el nombre de las columnas:

In [20]:
catalogues['RESULTADO'].columns

Index(['unnamed:_0', 'unnamed:_1'], dtype='object')

In [21]:
catalogues['RESULTADO'].columns = ['clave', 'descripción']
catalogues['RESULTADO'].columns

Index(['clave', 'descripción'], dtype='object')

2. Removemos la (primera) fila que no nos es útil.

In [22]:
catalogues['RESULTADO'] = catalogues['RESULTADO'].drop(index=0)

3. Convertimos el tipo de dato.

In [23]:
catalogues['RESULTADO']['clave'] = catalogues['RESULTADO']['clave'].astype(int)

> Fin del paréntesis. Hasta esta sección, esta parte de limpieza permite continuar con el proceso de manera normal.

In [24]:
# unimos otras columnnas que comparten lógica
other_cols = ['origen', 'sector', 'sexo', 'tipo_paciente', 'nacionalidad', 'resultado']
for col, cat in zip(other_cols, [x.upper() for x in other_cols]):
    data = (
        data.merge(catalogues[cat], how='left', left_on=col, right_on='clave').
        drop(columns=[col, 'clave']).
        rename(columns={'descripción':col})
    )

In [25]:
# creamos algunas variables que podrían ser útiles después
data['defuncion'] = ~data.fecha_def.isna()
data['mes_def'] = data.fecha_def.dt.month
data['año_def'] = data.fecha_def.dt.year
data['mes_ingreso'] = data.fecha_ingreso.dt.month
data['año_ingreso'] = data.fecha_ingreso.dt.year

In [27]:
data.head()

,fecha_actualizacion,id_registro,fecha_ingreso,fecha_sintomas,fecha_def,edad,pais_nacionalidad,pais_origen,entidad_res,municipio_res,entidad_um,entidad_nac,intubado,neumonia,embarazo,habla_lengua_indig,diabetes,epoc,asma,inmusupr,hipertension,otra_com,cardiovascular,obesidad,renal_cronica,tabaquismo,otro_caso,migrante,uci,origen,sector,sexo,tipo_paciente,nacionalidad,resultado,defuncion,mes_def,año_def,mes_ingreso,año_ingreso
0,2020-09-03,097d14,2020-06-10,2020-06-10,NaT,27,MÃ©xico,99,BAJA CALIFORNIA,MEXICALI,BAJA CALIFORNIA,BAJA CALIFORNIA,NO APLICA,NO,NO APLICA,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,SI,NO ESPECIFICADO,NO APLICA,FUERA DE USMER,ESTATAL,HOMBRE,AMBULATORIO,MEXICANA,Positivo SARS-CoV-2,False,NaN,NaN,6,2020
1,2020-09-03,1b7adb,2020-04-29,2020-04-15,2020-04-30,64,MÃ©xico,99,BAJA CALIFORNIA,TIJUANA,BAJA CALIFORNIA,GUERRERO,NO,SI,NO APLICA,NO,SI,NO,NO,NO,SI,NO,NO,NO,NO,NO,NO ESPECIFICADO,NO ESPECIFICADO,NO,FUERA DE USMER,IMSS,HOMBRE,HOSPITALIZADO,MEXICANA,Positivo SARS-CoV-2,True,4.0,2020.0,4,2020
2,2020-09-03,11daaa,2020-05-26,2020-05-26,NaT,32,MÃ©xico,99,TABASCO,TACOTALPA,TABASCO,TABASCO,NO APLICA,NO,NO,NO ESPECIFICADO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,SI,NO ESPECIFICADO,NO APLICA,FUERA DE USMER,ESTATAL,MUJER,AMBULATORIO,MEXICANA,Positivo SARS-CoV-2,False,NaN,NaN,5,2020
3,2020-09-03,17c9eb,2020-06-05,2020-05-31,NaT,54,MÃ©xico,99,TABASCO,NACAJUCA,TABASCO,TABASCO,NO APLICA,NO,NO,NO,SI,NO,NO,NO,SI,NO,NO,NO,NO,NO,SI,NO ESPECIFICADO,NO APLICA,FUERA DE USMER,ESTATAL,MUJER,AMBULATORIO,MEXICANA,Positivo SARS-CoV-2,False,NaN,NaN,6,2020
4,2020-09-03,14f12c,2020-04-02,2020-04-01,NaT,44,MÃ©xico,99,MORELOS,CUERNAVACA,MORELOS,BAJA CALIFORNIA SUR,NO APLICA,NO,NO,NO,NO,NO,NO,NO,NO,SI,NO,NO,NO,NO,SI,NO ESPECIFICADO,NO APLICA,FUERA DE USMER,IMSS,MUJER,AMBULATORIO,MEXICANA,Positivo SARS-CoV-2,False,NaN,NaN,4,2020


###Filtros

**Distintas formas de filtrar lo mismo**

In [28]:
# usando query
data.query('nacionalidad == "MEXICANA"').head(2)

,fecha_actualizacion,id_registro,fecha_ingreso,fecha_sintomas,fecha_def,edad,pais_nacionalidad,pais_origen,entidad_res,municipio_res,entidad_um,entidad_nac,intubado,neumonia,embarazo,habla_lengua_indig,diabetes,epoc,asma,inmusupr,hipertension,otra_com,cardiovascular,obesidad,renal_cronica,tabaquismo,otro_caso,migrante,uci,origen,sector,sexo,tipo_paciente,nacionalidad,resultado,defuncion,mes_def,año_def,mes_ingreso,año_ingreso
0,2020-09-03,097d14,2020-06-10,2020-06-10,NaT,27,MÃ©xico,99,BAJA CALIFORNIA,MEXICALI,BAJA CALIFORNIA,BAJA CALIFORNIA,NO APLICA,NO,NO APLICA,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,SI,NO ESPECIFICADO,NO APLICA,FUERA DE USMER,ESTATAL,HOMBRE,AMBULATORIO,MEXICANA,Positivo SARS-CoV-2,False,NaN,NaN,6,2020
1,2020-09-03,1b7adb,2020-04-29,2020-04-15,2020-04-30,64,MÃ©xico,99,BAJA CALIFORNIA,TIJUANA,BAJA CALIFORNIA,GUERRERO,NO,SI,NO APLICA,NO,SI,NO,NO,NO,SI,NO,NO,NO,NO,NO,NO ESPECIFICADO,NO ESPECIFICADO,NO,FUERA DE USMER,IMSS,HOMBRE,HOSPITALIZADO,MEXICANA,Positivo SARS-CoV-2,True,4.0,2020.0,4,2020


In [29]:
# haciendo referencia a la columna como índice
data[data['nacionalidad'] == 'MEXICANA'].head(2)

,fecha_actualizacion,id_registro,fecha_ingreso,fecha_sintomas,fecha_def,edad,pais_nacionalidad,pais_origen,entidad_res,municipio_res,entidad_um,entidad_nac,intubado,neumonia,embarazo,habla_lengua_indig,diabetes,epoc,asma,inmusupr,hipertension,otra_com,cardiovascular,obesidad,renal_cronica,tabaquismo,otro_caso,migrante,uci,origen,sector,sexo,tipo_paciente,nacionalidad,resultado,defuncion,mes_def,año_def,mes_ingreso,año_ingreso
0,2020-09-03,097d14,2020-06-10,2020-06-10,NaT,27,MÃ©xico,99,BAJA CALIFORNIA,MEXICALI,BAJA CALIFORNIA,BAJA CALIFORNIA,NO APLICA,NO,NO APLICA,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,SI,NO ESPECIFICADO,NO APLICA,FUERA DE USMER,ESTATAL,HOMBRE,AMBULATORIO,MEXICANA,Positivo SARS-CoV-2,False,NaN,NaN,6,2020
1,2020-09-03,1b7adb,2020-04-29,2020-04-15,2020-04-30,64,MÃ©xico,99,BAJA CALIFORNIA,TIJUANA,BAJA CALIFORNIA,GUERRERO,NO,SI,NO APLICA,NO,SI,NO,NO,NO,SI,NO,NO,NO,NO,NO,NO ESPECIFICADO,NO ESPECIFICADO,NO,FUERA DE USMER,IMSS,HOMBRE,HOSPITALIZADO,MEXICANA,Positivo SARS-CoV-2,True,4.0,2020.0,4,2020


In [30]:
# seleccionando directamente la columna que queremos filtrar
data[data.nacionalidad == 'MEXICANA'].head(2)

,fecha_actualizacion,id_registro,fecha_ingreso,fecha_sintomas,fecha_def,edad,pais_nacionalidad,pais_origen,entidad_res,municipio_res,entidad_um,entidad_nac,intubado,neumonia,embarazo,habla_lengua_indig,diabetes,epoc,asma,inmusupr,hipertension,otra_com,cardiovascular,obesidad,renal_cronica,tabaquismo,otro_caso,migrante,uci,origen,sector,sexo,tipo_paciente,nacionalidad,resultado,defuncion,mes_def,año_def,mes_ingreso,año_ingreso
0,2020-09-03,097d14,2020-06-10,2020-06-10,NaT,27,MÃ©xico,99,BAJA CALIFORNIA,MEXICALI,BAJA CALIFORNIA,BAJA CALIFORNIA,NO APLICA,NO,NO APLICA,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,SI,NO ESPECIFICADO,NO APLICA,FUERA DE USMER,ESTATAL,HOMBRE,AMBULATORIO,MEXICANA,Positivo SARS-CoV-2,False,NaN,NaN,6,2020
1,2020-09-03,1b7adb,2020-04-29,2020-04-15,2020-04-30,64,MÃ©xico,99,BAJA CALIFORNIA,TIJUANA,BAJA CALIFORNIA,GUERRERO,NO,SI,NO APLICA,NO,SI,NO,NO,NO,SI,NO,NO,NO,NO,NO,NO ESPECIFICADO,NO ESPECIFICADO,NO,FUERA DE USMER,IMSS,HOMBRE,HOSPITALIZADO,MEXICANA,Positivo SARS-CoV-2,True,4.0,2020.0,4,2020


**Filtrado múltiple**

In [31]:
# usando query podemos poner en el string del query todas las condiciones que queramos
data.query('nacionalidad == "MEXICANA" & entidad_res == "CIUDAD DE MÉXICO"').head(2)

,fecha_actualizacion,id_registro,fecha_ingreso,fecha_sintomas,fecha_def,edad,pais_nacionalidad,pais_origen,entidad_res,municipio_res,entidad_um,entidad_nac,intubado,neumonia,embarazo,habla_lengua_indig,diabetes,epoc,asma,inmusupr,hipertension,otra_com,cardiovascular,obesidad,renal_cronica,tabaquismo,otro_caso,migrante,uci,origen,sector,sexo,tipo_paciente,nacionalidad,resultado,defuncion,mes_def,año_def,mes_ingreso,año_ingreso
22,2020-09-03,192ec3,2020-04-23,2020-04-23,2020-05-03,42,MÃ©xico,99,CIUDAD DE MÉXICO,AZCAPOTZALCO,CIUDAD DE MÉXICO,CIUDAD DE MÉXICO,NO,SI,NO APLICA,NO,NO,NO,NO,NO,NO,NO,NO,SI,NO,NO,NO ESPECIFICADO,NO ESPECIFICADO,NO,FUERA DE USMER,IMSS,HOMBRE,HOSPITALIZADO,MEXICANA,Positivo SARS-CoV-2,True,5.0,2020.0,4,2020
26,2020-09-03,1be6c6,2020-05-19,2020-05-18,NaT,37,MÃ©xico,99,CIUDAD DE MÉXICO,AZCAPOTZALCO,CIUDAD DE MÉXICO,MICHOACÁN DE OCAMPO,NO APLICA,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO ESPECIFICADO,NO ESPECIFICADO,NO APLICA,FUERA DE USMER,IMSS,MUJER,AMBULATORIO,MEXICANA,Positivo SARS-CoV-2,False,NaN,NaN,5,2020


In [32]:
# usando el filtrado tradicional debemos encerrar en paréntesis cada condición que queremos que cumpla nuestro filtro
data[(data['nacionalidad'] == "MEXICANA") & (data.entidad_res == "CIUDAD DE MÉXICO")].head(2)

,fecha_actualizacion,id_registro,fecha_ingreso,fecha_sintomas,fecha_def,edad,pais_nacionalidad,pais_origen,entidad_res,municipio_res,entidad_um,entidad_nac,intubado,neumonia,embarazo,habla_lengua_indig,diabetes,epoc,asma,inmusupr,hipertension,otra_com,cardiovascular,obesidad,renal_cronica,tabaquismo,otro_caso,migrante,uci,origen,sector,sexo,tipo_paciente,nacionalidad,resultado,defuncion,mes_def,año_def,mes_ingreso,año_ingreso
22,2020-09-03,192ec3,2020-04-23,2020-04-23,2020-05-03,42,MÃ©xico,99,CIUDAD DE MÉXICO,AZCAPOTZALCO,CIUDAD DE MÉXICO,CIUDAD DE MÉXICO,NO,SI,NO APLICA,NO,NO,NO,NO,NO,NO,NO,NO,SI,NO,NO,NO ESPECIFICADO,NO ESPECIFICADO,NO,FUERA DE USMER,IMSS,HOMBRE,HOSPITALIZADO,MEXICANA,Positivo SARS-CoV-2,True,5.0,2020.0,4,2020
26,2020-09-03,1be6c6,2020-05-19,2020-05-18,NaT,37,MÃ©xico,99,CIUDAD DE MÉXICO,AZCAPOTZALCO,CIUDAD DE MÉXICO,MICHOACÁN DE OCAMPO,NO APLICA,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO ESPECIFICADO,NO ESPECIFICADO,NO APLICA,FUERA DE USMER,IMSS,MUJER,AMBULATORIO,MEXICANA,Positivo SARS-CoV-2,False,NaN,NaN,5,2020


###Resúmenes

In [33]:
# una columna, una métrica
data.groupby('sexo').edad.mean()

sexo
HOMBRE    42.812564
MUJER     41.735627
Name: edad, dtype: float64

In [34]:
# una columna, varias métricas
data.groupby('sexo').agg({'edad': ['min','mean','max']})

edad                
        min       mean  max
sexo                       
HOMBRE    0  42.812564  120
MUJER     0  41.735627  120

In [35]:
# varias columnas, una métrica
data.groupby('sexo').agg({'edad': 'mean', 'mes_ingreso': 'mean', 'mes_def': 'mean'})

,edad,mes_ingreso,mes_def
sexo,,,
HOMBRE,42.812564,6.548249,6.371355
MUJER,41.735627,6.611875,6.431488


In [36]:
# varias columnas, varias métricas
data.groupby('sexo').agg({'edad': ['min','mean','max'], 'mes_ingreso': ['min','mean','max'], 'mes_def': ['min','mean','max']})

edad                 mes_ingreso               mes_def               
        min       mean  max         min      mean max     min      mean  max
sexo                                                                        
HOMBRE    0  42.812564  120           1  6.548249   9     1.0  6.371355  9.0
MUJER     0  41.735627  120           1  6.611875   9     1.0  6.431488  9.0

###Descripción por Tipo

In [37]:
# Filtrar por tipos generales (categóricas y numéricas)
numerical = data.select_dtypes(include='number')
categorical = data.select_dtypes(include=['object', 'bool', 'category', 'datetime64[ns]'])

In [38]:
numerical.head()

,edad,mes_def,año_def,mes_ingreso,año_ingreso
0,27,NaN,NaN,6,2020
1,64,4.0,2020.0,4,2020
2,32,NaN,NaN,5,2020
3,54,NaN,NaN,6,2020
4,44,NaN,NaN,4,2020


In [39]:
categorical.head()

,fecha_actualizacion,id_registro,fecha_ingreso,fecha_sintomas,fecha_def,pais_nacionalidad,pais_origen,entidad_res,municipio_res,entidad_um,entidad_nac,intubado,neumonia,embarazo,habla_lengua_indig,diabetes,epoc,asma,inmusupr,hipertension,otra_com,cardiovascular,obesidad,renal_cronica,tabaquismo,otro_caso,migrante,uci,origen,sector,sexo,tipo_paciente,nacionalidad,resultado,defuncion
0,2020-09-03,097d14,2020-06-10,2020-06-10,NaT,MÃ©xico,99,BAJA CALIFORNIA,MEXICALI,BAJA CALIFORNIA,BAJA CALIFORNIA,NO APLICA,NO,NO APLICA,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,SI,NO ESPECIFICADO,NO APLICA,FUERA DE USMER,ESTATAL,HOMBRE,AMBULATORIO,MEXICANA,Positivo SARS-CoV-2,False
1,2020-09-03,1b7adb,2020-04-29,2020-04-15,2020-04-30,MÃ©xico,99,BAJA CALIFORNIA,TIJUANA,BAJA CALIFORNIA,GUERRERO,NO,SI,NO APLICA,NO,SI,NO,NO,NO,SI,NO,NO,NO,NO,NO,NO ESPECIFICADO,NO ESPECIFICADO,NO,FUERA DE USMER,IMSS,HOMBRE,HOSPITALIZADO,MEXICANA,Positivo SARS-CoV-2,True
2,2020-09-03,11daaa,2020-05-26,2020-05-26,NaT,MÃ©xico,99,TABASCO,TACOTALPA,TABASCO,TABASCO,NO APLICA,NO,NO,NO ESPECIFICADO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,SI,NO ESPECIFICADO,NO APLICA,FUERA DE USMER,ESTATAL,MUJER,AMBULATORIO,MEXICANA,Positivo SARS-CoV-2,False
3,2020-09-03,17c9eb,2020-06-05,2020-05-31,NaT,MÃ©xico,99,TABASCO,NACAJUCA,TABASCO,TABASCO,NO APLICA,NO,NO,NO,SI,NO,NO,NO,SI,NO,NO,NO,NO,NO,SI,NO ESPECIFICADO,NO APLICA,FUERA DE USMER,ESTATAL,MUJER,AMBULATORIO,MEXICANA,Positivo SARS-CoV-2,False
4,2020-09-03,14f12c,2020-04-02,2020-04-01,NaT,MÃ©xico,99,MORELOS,CUERNAVACA,MORELOS,BAJA CALIFORNIA SUR,NO APLICA,NO,NO,NO,NO,NO,NO,NO,NO,SI,NO,NO,NO,NO,SI,NO ESPECIFICADO,NO APLICA,FUERA DE USMER,IMSS,MUJER,AMBULATORIO,MEXICANA,Positivo SARS-CoV-2,False


In [40]:
# para conocer todos los tipo de datos
?data.select_dtypes

####Categóricas

In [41]:
# Cardinalidad - Valores únicos por columna
for col in categorical.columns:
  print('Valores únicos en "{}": {}'.format(col,categorical[col].nunique()))

Valores únicos en "fecha_actualizacion": 1
Valores únicos en "id_registro": 1393033
Valores únicos en "fecha_ingreso": 247
Valores únicos en "fecha_sintomas": 247
Valores únicos en "fecha_def": 192
Valores únicos en "pais_nacionalidad": 119
Valores únicos en "pais_origen": 78
Valores únicos en "entidad_res": 32
Valores únicos en "municipio_res": 2155
Valores únicos en "entidad_um": 32
Valores únicos en "entidad_nac": 33
Valores únicos en "intubado": 4
Valores únicos en "neumonia": 3
Valores únicos en "embarazo": 4
Valores únicos en "habla_lengua_indig": 3
Valores únicos en "diabetes": 3
Valores únicos en "epoc": 3
Valores únicos en "asma": 3
Valores únicos en "inmusupr": 3
Valores únicos en "hipertension": 3
Valores únicos en "otra_com": 3
Valores únicos en "cardiovascular": 3
Valores únicos en "obesidad": 3
Valores únicos en "renal_cronica": 3
Valores únicos en "tabaquismo": 3
Valores únicos en "otro_caso": 3
Valores únicos en "migrante": 3
Valores únicos en "uci": 4
Valores únicos en

In [42]:
# Unicidad - proporción de valores únicos por columna
total = data.shape[0]
for col in categorical.columns:
  print('Porcentaje de valores únicos en "{}": {}%'.format(col,round(categorical[col].nunique()/total*100,2)))

Porcentaje de valores únicos en "fecha_actualizacion": 0.0%
Porcentaje de valores únicos en "id_registro": 100.0%
Porcentaje de valores únicos en "fecha_ingreso": 0.02%
Porcentaje de valores únicos en "fecha_sintomas": 0.02%
Porcentaje de valores únicos en "fecha_def": 0.01%
Porcentaje de valores únicos en "pais_nacionalidad": 0.01%
Porcentaje de valores únicos en "pais_origen": 0.01%
Porcentaje de valores únicos en "entidad_res": 0.0%
Porcentaje de valores únicos en "municipio_res": 0.15%
Porcentaje de valores únicos en "entidad_um": 0.0%
Porcentaje de valores únicos en "entidad_nac": 0.0%
Porcentaje de valores únicos en "intubado": 0.0%
Porcentaje de valores únicos en "neumonia": 0.0%
Porcentaje de valores únicos en "embarazo": 0.0%
Porcentaje de valores únicos en "habla_lengua_indig": 0.0%
Porcentaje de valores únicos en "diabetes": 0.0%
Porcentaje de valores únicos en "epoc": 0.0%
Porcentaje de valores únicos en "asma": 0.0%
Porcentaje de valores únicos en "inmusupr": 0.0%
Porcenta

In [43]:
# Moda - valor que más se repite en cada columna excepto los indicadores únicos
for col in categorical.drop(columns='id_registro').columns:
  print('Valor más popular en "{}": {}'.format(col, statistics.mode(categorical[col])))

Valor más popular en "fecha_actualizacion": 2020-09-03 00:00:00
Valor más popular en "fecha_ingreso": 2020-07-20 00:00:00
Valor más popular en "fecha_sintomas": 2020-07-20 00:00:00
Valor más popular en "fecha_def": NaT
Valor más popular en "pais_nacionalidad": MÃ©xico
Valor más popular en "pais_origen": 99
Valor más popular en "entidad_res": CIUDAD DE MÉXICO
Valor más popular en "municipio_res": IZTAPALAPA
Valor más popular en "entidad_um": CIUDAD DE MÉXICO
Valor más popular en "entidad_nac": CIUDAD DE MÉXICO
Valor más popular en "intubado": NO APLICA
Valor más popular en "neumonia": NO 
Valor más popular en "embarazo": NO 
Valor más popular en "habla_lengua_indig": NO 
Valor más popular en "diabetes": NO 
Valor más popular en "epoc": NO 
Valor más popular en "asma": NO 
Valor más popular en "inmusupr": NO 
Valor más popular en "hipertension": NO 
Valor más popular en "otra_com": NO 
Valor más popular en "cardiovascular": NO 
Valor más popular en "obesidad": NO 
Valor más popular en "r

####Numéricas

In [44]:
numerical.describe()

,edad,mes_def,año_def,mes_ingreso,año_ingreso
count,1.393033e+06,84677.000000,84677.0,1.393033e+06,1393033.0
mean,4.226682e+01,6.393200,2020.0,6.580492e+00,2020.0
std,1.678013e+01,1.218526,0.0,1.288985e+00,0.0
min,0.000000e+00,1.000000,2020.0,1.000000e+00,2020.0
25%,3.000000e+01,5.000000,2020.0,6.000000e+00,2020.0
50%,4.100000e+01,6.000000,2020.0,7.000000e+00,2020.0
75%,5.300000e+01,7.000000,2020.0,8.000000e+00,2020.0
max,1.200000e+02,9.000000,2020.0,9.000000e+00,2020.0


##Análisis Exploratorio

En esta sección responderemos algunas preguntas sobre el set de datos usando las herramientas que hemos visto hasta ahora

In [45]:
# ¿Cuántas pruebas por origen se realizan?
data.groupby('origen').id_registro.count()

origen
FUERA DE USMER    928460
USMER             464573
Name: id_registro, dtype: int64

In [46]:
data.origen.value_counts()

FUERA DE USMER    928460
USMER             464573
Name: origen, dtype: int64

In [47]:
# ¿Cuántas pruebas por sector se realizan?
data.sector.value_counts()

SSA                863605
IMSS               359287
ISSSTE              52959
PRIVADA             52236
ESTATAL             25729
PEMEX               12225
IMSS-BIENESTAR       9690
SEMAR                7387
SEDENA               6890
MUNICIPAL            1128
UNIVERSITARIO         897
DIF                   786
CRUZ ROJA             206
NO ESPECIFICADO         8
Name: sector, dtype: int64

In [48]:
# ¿Cuántos pacientes por sexo tenemos?
data.sexo.value_counts()

MUJER     705922
HOMBRE    687111
Name: sexo, dtype: int64

In [49]:
# ¿Cuál es la proporción de los pacientes por sexo?
data.sexo.value_counts(normalize=True)

MUJER     0.506752
HOMBRE    0.493248
Name: sexo, dtype: float64

In [50]:
# ¿Cuál es la proporción de resultados de las pruebas?
data.resultado.value_counts(normalize=True)

No positivo SARS-CoV-2    0.496987
Positivo SARS-CoV-2       0.442842
Resultado pendiente       0.060171
Name: resultado, dtype: float64

In [51]:
# ¿Cuántas pruebas positivas tenemos en el set?
data[data.resultado == 'Positivo SARS-CoV-2'].id_registro.count()

616894

In [52]:
# ¿Cuántos casos positivos tenemos por mes?
data[data.resultado == 'Positivo SARS-CoV-2'].mes_ingreso.value_counts().sort_index()

1         2
2         8
3      2530
4     26554
5     86466
6    152756
7    198622
8    148077
9      1879
Name: mes_ingreso, dtype: int64

In [53]:
# ¿Cuál es el resultado de las pruebas por sexo?
data.groupby(['sexo','resultado'])['id_registro'].count()

sexo    resultado             
HOMBRE  No positivo SARS-CoV-2    322529
        Positivo SARS-CoV-2       323257
        Resultado pendiente        41325
MUJER   No positivo SARS-CoV-2    369790
        Positivo SARS-CoV-2       293637
        Resultado pendiente        42495
Name: id_registro, dtype: int64

In [54]:
data.groupby('sexo').resultado.value_counts()

sexo    resultado             
HOMBRE  Positivo SARS-CoV-2       323257
        No positivo SARS-CoV-2    322529
        Resultado pendiente        41325
MUJER   No positivo SARS-CoV-2    369790
        Positivo SARS-CoV-2       293637
        Resultado pendiente        42495
Name: resultado, dtype: int64

In [55]:
# ¿Qué proporción de resultados de pruebas tenemos por sexo?
data.groupby('sexo').resultado.value_counts(normalize=True)

sexo    resultado             
HOMBRE  Positivo SARS-CoV-2       0.470458
        No positivo SARS-CoV-2    0.469399
        Resultado pendiente       0.060143
MUJER   No positivo SARS-CoV-2    0.523840
        Positivo SARS-CoV-2       0.415962
        Resultado pendiente       0.060198
Name: resultado, dtype: float64

In [56]:
# ¿Cuál es la edad de los pacientes por sexo?
data.groupby('sexo').edad.describe()

,count,mean,std,min,25%,50%,75%,max
sexo,,,,,,,,
HOMBRE,687111.0,42.812564,17.091827,0.0,30.0,41.0,54.0,120.0
MUJER,705922.0,41.735627,16.453715,0.0,30.0,40.0,52.0,120.0


In [57]:
def q25(x):
  return x.quantile(0.25)

def q75(x):
  return x.quantile(0.75)

data.groupby('sexo').agg({'edad': ['min', q25, 'median', q75, 'max']})

edad                    
        min q25 median q75  max
sexo                           
HOMBRE    0  30     41  54  120
MUJER     0  30     40  52  120

In [58]:
# ¿Cuál es la edad de los pacientes dependiendo de si son intubados o no?
data.groupby('intubado').edad.describe()

,count,mean,std,min,25%,50%,75%,max
intubado,,,,,,,,
NO,219583.0,53.630053,19.435595,0.0,42.0,55.0,67.0,120.0
NO APLICA,1144649.0,39.748604,15.020323,0.0,29.0,38.0,49.0,120.0
NO ESPECIFICADO,167.0,53.005988,18.042619,0.0,41.5,54.0,65.5,92.0
SI,28634.0,55.730286,18.850803,0.0,47.0,58.0,68.0,108.0


In [59]:
data.groupby('intubado').agg({'edad': ['min', q25, 'median', q75, 'max']})

edad                        
                 min   q25 median   q75  max
intubado                                    
NO                 0  42.0     55  67.0  120
NO APLICA          0  29.0     38  49.0  120
NO ESPECIFICADO    0  41.5     54  65.5   92
SI                 0  47.0     58  68.0  108

In [60]:
# ¿Cuál es la edad máxima y cuál es la mínima de los pacientes fallecidos?
edad_def_max = data[data.defuncion].edad.max()
data[data.defuncion & (data.edad == edad_def_max)]

,fecha_actualizacion,id_registro,fecha_ingreso,fecha_sintomas,fecha_def,edad,pais_nacionalidad,pais_origen,entidad_res,municipio_res,entidad_um,entidad_nac,intubado,neumonia,embarazo,habla_lengua_indig,diabetes,epoc,asma,inmusupr,hipertension,otra_com,cardiovascular,obesidad,renal_cronica,tabaquismo,otro_caso,migrante,uci,origen,sector,sexo,tipo_paciente,nacionalidad,resultado,defuncion,mes_def,año_def,mes_ingreso,año_ingreso
551494,2020-09-03,1b7b20,2020-05-21,2020-05-20,2020-05-26,109,MÃ©xico,99,MICHOACÁN DE OCAMPO,APATZINGÁN,MICHOACÁN DE OCAMPO,MICHOACÁN DE OCAMPO,NO,SI,NO APLICA,NO,NO,NO,NO,SI,NO,NO,NO,NO,SI,NO,NO,NO ESPECIFICADO,NO,FUERA DE USMER,ISSSTE,HOMBRE,HOSPITALIZADO,MEXICANA,No positivo SARS-CoV-2,True,5.0,2020.0,5,2020


In [61]:
edad_def_min = data[data.defuncion].edad.min()
data[data.defuncion & (data.edad == edad_def_min)]

,fecha_actualizacion,id_registro,fecha_ingreso,fecha_sintomas,fecha_def,edad,pais_nacionalidad,pais_origen,entidad_res,municipio_res,entidad_um,entidad_nac,intubado,neumonia,embarazo,habla_lengua_indig,diabetes,epoc,asma,inmusupr,hipertension,otra_com,cardiovascular,obesidad,renal_cronica,tabaquismo,otro_caso,migrante,uci,origen,sector,sexo,tipo_paciente,nacionalidad,resultado,defuncion,mes_def,año_def,mes_ingreso,año_ingreso
8162,2020-09-03,15f637,2020-06-25,2020-06-20,2020-07-01,0,MÃ©xico,99,GUANAJUATO,LEÓN,GUANAJUATO,GUANAJUATO,NO,NO,NO APLICA,NO,NO,NO,NO,NO,SI,NO,NO,NO,NO,NO,NO,NO ESPECIFICADO,NO,FUERA DE USMER,SSA,HOMBRE,HOSPITALIZADO,MEXICANA,Positivo SARS-CoV-2,True,7.0,2020.0,6,2020
16059,2020-09-03,083c34,2020-05-25,2020-05-25,2020-06-01,0,MÃ©xico,99,MÉXICO,CHIMALHUACÁN,MÉXICO,MÉXICO,SI,SI,NO APLICA,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,SI,NO ESPECIFICADO,SI,USMER,IMSS,HOMBRE,HOSPITALIZADO,MEXICANA,Positivo SARS-CoV-2,True,6.0,2020.0,5,2020
23799,2020-09-03,02f8e2,2020-05-08,2020-05-01,2020-05-10,0,MÃ©xico,99,BAJA CALIFORNIA,TIJUANA,BAJA CALIFORNIA,BAJA CALIFORNIA,SI,SI,NO APLICA,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO ESPECIFICADO,NO,USMER,SSA,HOMBRE,HOSPITALIZADO,MEXICANA,Positivo SARS-CoV-2,True,5.0,2020.0,5,2020
46287,2020-09-03,1cc0ea,2020-04-24,2020-04-21,2020-04-29,0,MÃ©xico,99,MÉXICO,ECATEPEC DE MORELOS,MÉXICO,MÉXICO,NO,SI,NO,NO,SI,NO,NO,NO,SI,NO,NO,NO,NO,NO,NO ESPECIFICADO,NO ESPECIFICADO,NO,FUERA DE USMER,SSA,MUJER,HOSPITALIZADO,MEXICANA,Positivo SARS-CoV-2,True,4.0,2020.0,4,2020
47645,2020-09-03,014b5d,2020-05-18,2020-05-17,2020-05-18,0,MÃ©xico,99,BAJA CALIFORNIA,TIJUANA,BAJA CALIFORNIA,BAJA CALIFORNIA,SI,SI,NO APLICA,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO ESPECIFICADO,NO,FUERA DE USMER,ISSSTE,HOMBRE,HOSPITALIZADO,MEXICANA,Positivo SARS-CoV-2,True,5.0,2020.0,5,2020
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1323007,2020-09-03,079171,2020-07-14,2020-07-14,2020-07-18,0,MÃ©xico,99,CHIAPAS,ESCUINTLA,CHIAPAS,CHIAPAS,SI,SI,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,SI,NO ESPECIFICADO,NO,USMER,SSA,MUJER,HOSPITALIZADO,MEXICANA,No positivo SARS-CoV-2,True,7.0,2020.0,7,2020
1325181,2020-09-03,0860f4,2020-06-30,2020-06-30,2020-07-13,0,MÃ©xico,99,PUEBLA,TEHUACÁN,PUEBLA,PUEBLA,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO ESPECIFICADO,NO ESPECIFICADO,NO,FUERA DE USMER,IMSS,MUJER,HOSPITALIZADO,MEXICANA,Resultado pendiente,True,7.0,2020.0,6,2020
1335704,2020-09-03,1e7d7f,2020-08-18,2020-08-15,2020-09-01,0,MÃ©xico,99,JALISCO,TONALÁ,JALISCO,JALISCO,NO,SI,NO APLICA,NO ESPECIFICADO,NO,NO,NO,NO,NO,SE IGNORA,NO,NO,NO,NO,NO,NO ESPECIFICADO,NO,USMER,SSA,HOMBRE,HOSPITALIZADO,MEXICANA,Resultado pendiente,True,9.0,2020.0,8,2020
1356918,2020-09-03,00f39c,2020-07-06,2020-07-03,2020-07-08,0,MÃ©xico,99,JALISCO,ZAPOTILTIC,JALISCO,JALISCO,NO,SI,NO APLICA,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO ESPECIFICADO,SI,USMER,SSA,HOMBRE,HOSPITALIZADO,MEXICANA,Resultado pendiente,True,7.0,2020.0,7,2020


In [62]:
# ¿Cuál es la edad de los pacientes que fallecen?
data.groupby('defuncion').edad.describe()

,count,mean,std,min,25%,50%,75%,max
defuncion,,,,,,,,
False,1308356.0,41.001587,16.077933,0.0,30.0,40.0,51.0,120.0
True,84677.0,61.816172,15.204152,0.0,53.0,63.0,72.0,109.0


In [63]:
# ¿Cuántas defunciones tenemos por mes?
data[data.defuncion].mes_def.value_counts().sort_index()

1.0        6
2.0       12
3.0      232
4.0     5277
5.0    16018
6.0    21318
7.0    23471
8.0    17859
9.0      484
Name: mes_def, dtype: int64

In [64]:
# ¿Cuál es la proporción de diábeticos para las defunciones?
data.groupby('diabetes').defuncion.value_counts(normalize=True)

diabetes   defuncion
NO         False        0.957313
           True         0.042687
SE IGNORA  False        0.873791
           True         0.126209
SI         False        0.812020
           True         0.187980
Name: defuncion, dtype: float64

In [65]:
# ¿Cuál es la proporción de asmáticos para las defunciones?
data.groupby('asma').defuncion.value_counts(normalize=True)

asma       defuncion
NO         False        0.938871
           True         0.061129
SE IGNORA  False        0.865200
           True         0.134800
SI         False        0.957209
           True         0.042791
Name: defuncion, dtype: float64